# LA PREMIERE CELL EST POUR TE MONTRER COMMENT LES TRANSFORMATIONS FONCTIONNENT ET COMMENT L'ENVIRONNEMENT EST MODIFIE

In [ ]:
import gymnasium as gym
from sb3_contrib import RecurrentPPO  # PPO récurrent (LSTM)
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from gymnasium.wrappers import ResizeObservation, GrayscaleObservation
import torch.nn as nn

# référence pour le dictionnaire d'hyperparamètre et des transformations: https://huggingface.co/sb3/ppo_lstm-CarRacing-v0

"""
J'UTILISE DUMMYVECENV ET VECFRAMESTACK CAR LE FrameStackObservation fait bug, chatgpt a proposé cette alternative
Voici sa réponse:
"Le problème vient du fait que Stable-Baselines3 (SB3) et ses algorithmes (y compris RecurrentPPO) s'attendent généralement à un environnement 
vectorisé et utilisent habituellement la classe VecFrameStack pour empiler les observations, plutôt qu'un wrapper Gymnasium standard comme FrameStackObservation

SB3 fonctionne de façon optimale avec un VecEnv (environnement vectorisé), par exemple DummyVecEnv ou SubprocVecEnv

L'empilement d'images se fait alors via VecFrameStack (fournie par SB3) plutôt qu'un wrapper Gymnasium 
(qui n'est pas forcément compatible avec la vérification d'espace d'observation qu'effectue SB3)"
"""

def make_env():
    def _init():
        env = gym.make("CarRacing-v3", continuous=True, render_mode="rgb_array")
        env = ResizeObservation(env, (64, 64))
        env = GrayscaleObservation(env, keep_dim=True)
        return env
    return _init

print("Loading CarRacing-v3 environment")
env = gym.make("CarRacing-v3", continuous=True, render_mode="rgb_array")
print("Observation space size:", env.observation_space.shape)
env = ResizeObservation(env, (64, 64)) # 64 from the dict
print("Observation space resize:", env.observation_space.shape)
print("Observation space:", env.observation_space)
env = GrayscaleObservation(env, keep_dim = True)
print("Observation space after gray scaling:", env.observation_space)

n_envs = 8
env = DummyVecEnv([make_env() for _ in range(n_envs)])
env = VecFrameStack(env, n_stack=2)
print(env.observation_space)

Loading CarRacing-v3 environment
Observation space size: (96, 96, 3)
Observation space resize: (64, 64, 3)
Observation space: Box(0, 255, (64, 64, 3), uint8)
Observation space after gray scaling: Box(0, 255, (64, 64, 1), uint8)
Observation space size after frame stacking: Box(0, 255, (64, 64, 1), uint8)
Box(0, 255, (64, 64, 2), uint8)


In [ ]:
import gymnasium as gym
from sb3_contrib import RecurrentPPO  # PPO récurrent (LSTM)
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from gymnasium.wrappers import ResizeObservation, GrayscaleObservation
import torch.nn as nn

# référence pour le dictionnaire d'hyperparamètre et des transformations: https://huggingface.co/sb3/ppo_lstm-CarRacing-v0

"""
DANS LE DICT IL Y A ENCORE
('learning_rate', 'lin_1e-4') => faut il faire autre chose que 1e-4 pour le learning rate ?
('normalize', "{'norm_obs': False, 'norm_reward': True}") => AUCUNE IDEE DE A QUOI CA FAIT REFERENCE
('normalize_kwargs', {'norm_obs': False, 'norm_reward': False})] => AUCUNE IDEE DE A QUOI CA FAIT REFERENCE

J'UTILISE DUMMYVECENV ET VECFRAMESTACK CAR LE FrameStackObservation fait bug, chatgpt a proposé cette alternative
Voici sa réponse:
"Le problème vient du fait que Stable-Baselines3 (SB3) et ses algorithmes (y compris RecurrentPPO) s'attendent généralement à un environnement 
vectorisé et utilisent habituellement la classe VecFrameStack pour empiler les observations, plutôt qu'un wrapper Gymnasium standard comme FrameStackObservation

SB3 fonctionne de façon optimale avec un VecEnv (environnement vectorisé), par exemple DummyVecEnv ou SubprocVecEnv

L'empilement d'images se fait alors via VecFrameStack (fournie par SB3) plutôt qu'un wrapper Gymnasium 
(qui n'est pas forcément compatible avec la vérification d'espace d'observation qu'effectue SB3)"
"""

def make_env():
    def _init():
        env = gym.make("CarRacing-v3", continuous=True, render_mode="rgb_array") # entrainement sur "rgb-array" car plus opti en terme de compute, le mode human display qqch
        env = ResizeObservation(env, (64, 64))
        env = GrayscaleObservation(env, keep_dim=True)
        return env
    return _init

print("Loading CarRacing-v3 environment")
n_envs = 8
env = DummyVecEnv([make_env() for _ in range(n_envs)])
env = VecFrameStack(env, n_stack=2)

# Dictionnaire policy kwargs venant du dict
policy_kwargs = {
    "log_std_init": -2,
    "ortho_init": False,
    "enable_critic_lstm": False,
    "activation_fn": nn.GELU,
    "lstm_hidden_size": 128,
}

# Dictionnaire d'hyperparamètres venant du dict
hyperparams = {
    "batch_size": 128,
    "clip_range": 0.2,
    "ent_coef": 0.0,
    "gae_lambda": 0.95,
    "gamma": 0.99,
    "learning_rate": 1e-4, # pas sur de ça car dans le dict c'est 'lin_1e-4'
    "max_grad_norm": 0.5,
    "n_epochs": 10,
    "n_steps": 512,
    "sde_sample_freq": 4,
    "use_sde": True,
    "vf_coef": 0.5,
    "policy_kwargs": policy_kwargs
}

# Instanciation du modèle RecurrentPPO avec la politique 'CnnLstmPolicy'
print("Doing PPOLSTM")
model = RecurrentPPO("CnnLstmPolicy", env, **hyperparams)

# Nombre total de timesteps pour l'entraînement (n_timesteps = 4e6), je pense ça fait référence à n_timesteps dans le dict
total_timesteps = 4000000.0

# Lancement de l'entraînement
# print("Start training")
model.learn(total_timesteps=total_timesteps, progress_bar=True) # le learning fonctionne bien

# # Sauvegarde du modèle final
# model.save("ppo_lstm_carracing_final")

Loading CarRacing-v3 environment
Doing PPOLSTM


Output()

: 